In [1]:
import imgaug as ia
import imageio
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import os
import xml.etree.ElementTree as ET
from PIL import Image

sometimes = lambda aug: iaa.Sometimes(0.5, aug)
seq = iaa.Sequential([
    iaa.Add((-15,30)),
    iaa.AdditiveGaussianNoise(scale=0.01*255,per_channel=True),
    #iaa.PiecewiseAffine(scale=(0.001, 0.003)),
    sometimes(iaa.Crop(percent=(0, 0.1))),
    iaa.SomeOf((0, 5),
            [
                # Convert some images into their superpixel representation,
                # sample between 20 and 200 superpixels per image, but do
                # not replace all superpixels with their average, only
                # some of them (p_replace).
                sometimes(
                    iaa.Superpixels(
                        p_replace=(0, 1.0),
                        n_segments=(20, 200)
                    )
                ),

                # Blur each image with varying strength using
                # gaussian blur (sigma between 0 and 3.0),
                # average/uniform blur (kernel size between 2x2 and 7x7)
                # median blur (kernel size between 3x3 and 11x11).
                iaa.OneOf([
                    iaa.GaussianBlur((0, 1.5)),
                    iaa.AverageBlur(k=(2, 3)),
                    iaa.MedianBlur(k=(1, 3)),
                ]),

                # Sharpen each image, overlay the result with the original
                # image using an alpha between 0 (no sharpening) and 1
                # (full sharpening effect).
                iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),

                # Same as sharpen, but for an embossing effect.
                iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),

                # Search in some images either for all edges or for
                # directed edges. These edges are then marked in a black
                # and white image and overlayed with the original image
                # using an alpha of 0 to 0.7.
                sometimes(iaa.OneOf([
                    iaa.EdgeDetect(alpha=(0, 0.7)),
                    iaa.DirectedEdgeDetect(
                        alpha=(0, 0.7), direction=(0.0, 1.0)
                                            ),
                ]))]),
    # Invert each image's channel with 5% probability.
    # This sets each pixel value v to 255-v.
    #iaa.Invert(0.05, per_channel=True), # invert color channels

    # Add a value of -10 to 10 to each pixel.
    #iaa.Add((-10, 10), per_channel=0.5),

    # Change brightness of images (50-150% of original value).
    iaa.Multiply((0.5, 1.5), per_channel=0.5),

    # Improve or worsen the contrast of images.
    iaa.LinearContrast((0.5, 2.0), per_channel=0.5),

    # Convert each image to grayscale and then overlay the
    # result with the original with random alpha. I.e. remove
    # colors with varying strengths.
    #iaa.Grayscale(alpha=(0.0, 1.0)),
    #sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))),
    iaa.Fliplr(0.5), # horizontally flip 50% of all images
    iaa.Flipud(0.2),
    iaa.Multiply((0.5, 1.5), per_channel=0.5),
    iaa.Affine(scale=(0.8,1.2),rotate=(-45,45),shear=(-30,30))
],random_order=True)
"""
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(
        0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    # Strengthen or weaken the contrast in each image.
    iaa.LinearContrast((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True)"""

datapath = "./trashV2/JPEGImages/"
xmlpath = "./trashV2/Annotations/"
count = 0
datalist = os.listdir(datapath)
for imagename in datalist:
    if ".jpg" in imagename:
        count +=1
        image = imageio.imread(os.path.join(datapath,imagename))
        print(imagename,len(datalist)-count)
        xml = ET.parse(os.path.join(xmlpath,imagename.split(".")[0]+".xml"))
        root = xml.getroot()
        objects = root.findall("object")
        boundingBoxList = []
        xmin = []
        xmax = []
        ymin = []
        ymax = []
        for index, item in enumerate(objects):
            boundingBox = item.find("bndbox")
            xmin.append(boundingBox.find("xmin"))
            xmax.append(boundingBox.find("xmax"))
            ymin.append(boundingBox.find("ymin"))
            ymax.append(boundingBox.find("ymax"))
            boundingBoxList.append(BoundingBox(x1=int(xmin[index].text),
                                               x2=int(xmax[index].text),
                                               y1=int(ymin[index].text),
                                               y2=int(ymax[index].text)))
        boundingBoxes = BoundingBoxesOnImage(boundingBoxList,shape=(image.shape[0],image.shape[1]))
        augedImage, augedBoxes = seq(image=image,bounding_boxes=boundingBoxes)
        for index, box in enumerate(augedBoxes):
            xmin[index].text = str(int(box.x1))
            xmax[index].text = str(int(box.x2))
            ymin[index].text = str(int(box.y1))
            ymax[index].text = str(int(box.y2))
        newname = imagename.split(".")[0]+"_Aug1.jpg"
        img = Image.fromarray(augedImage).convert('RGB')
        img.save(os.path.join(datapath,newname))
        xml.write(os.path.join(xmlpath,newname.split(".")[0]+".xml"))



KeyboardInterrupt: 

In [2]:
import os
count = 0
for item in os.listdir("../../data/trashV2/JPEGImages"):
    if "test" in item and "jpg" in item:
        count+=1
print(count)

192


In [2]:
import os
datadir = "trashV2/JPEGImages"
xmldir = "trashV2/Annotations"
count = 1
import xml.etree.ElementTree as ET
for item in os.listdir(datadir):
    newname = "image_"+str(count)+".jpg"
    xmlname = item.split(".")[0]+".xml"
    tree = ET.parse(os.path.join(xmldir,xmlname))
    root = tree.getroot()
    filename = root.find("filename")
    print(filename.text)
    filename.text = newname
    tree.write(os.path.join(xmldir,xmlname))
    newxmlname = newname.split(".")[0]+".xml"
    print(newxmlname)
    count +=1
    os.rename(os.path.join(datadir,item),os.path.join(datadir,newname))
    os.rename(os.path.join(xmldir,xmlname),os.path.join(xmldir,newxmlname))

FileNotFoundError: [Errno 2] No such file or directory: 'trashV2/JPEGImages'

In [32]:
import imgaug.augmenters as iaa
aug = iaa.PiecewiseAffine(scale=(0.15))

In [33]:
import imageio
import imgaug as ia
%matplotlib inline

image = imageio.imread("/Users/yuanlin/Desktop/image_1.jpg")
new_image = aug(image=image)

In [34]:
imageio.imwrite("/Users/yuanlin/Desktop/result.jpg",new_image)